In [1]:
# NUM_CLASSES = 2

# # Fixed for Cats & Dogs color images
# CHANNELS = 3

# IMAGE_RESIZE = 256
# RESNET50_POOLING_AVERAGE = 'avg'
# DENSE_LAYER_ACTIVATION = 'softmax'
# OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# # Common accuracy metric for all outputs, but can use different metrics for different output
# LOSS_METRICS = ['accuracy']

# # EARLY_STOP_PATIENCE must be < NUM_EPOCHS
# NUM_EPOCHS = 100
# EARLY_STOP_PATIENCE = 10

# STEPS_PER_EPOCH_TRAINING = 10
# STEPS_PER_EPOCH_VALIDATION = 10

# # These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# # NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
# BATCH_SIZE_TRAINING = 100
# BATCH_SIZE_VALIDATION = 100

# # Using 1 to easily manage mapping between test_generator & prediction for submission preparation
# BATCH_SIZE_TESTING = 1



# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense

# #resnet_weights_path =  r"D:\ML\ML-codes\resnet50_weights_.h5"
# model = Sequential()

# # 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# # NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
# model.add(ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights = 'imagenet',input_shape=(256,256,3)))

# # 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
# model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))

# # Say not to train first layer (ResNet) model as it is already trained
# model.layers[0].trainable = False




# from tensorflow.keras import optimizers

# sgd = optimizers.SGD(learning_rate = 0.01,  momentum = 0.9, nesterov = True)
# model.compile(optimizer = sgd, loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)



# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)
# cb_checkpointer = ModelCheckpoint(filepath = r'/workspace/resnet50.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')


# fit_history = model.fit_generator(
#         train_generator,
#         steps_per_epoch=STEPS_PER_EPOCH_TRAINING,
#         epochs = NUM_EPOCHS,
#         validation_data=val_generator,
#         validation_steps=STEPS_PER_EPOCH_VALIDATION,
#         callbacks=[cb_checkpointer, cb_early_stopper]
# )

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import 
import keras
from keras.layers import *
from keras.layers import *
from keras.preprocessing import image

In [3]:
# gpus = tf.config.experimental.list_physical_devices('GPU')

# if gpus:
#     tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(f"Found {len(gpus)} GPU(s) and {len(logical_gpus)} logical GPU(s).")
# else:
#     print("No GPU(s) found. Using CPU.")

No GPU(s) found. Using CPU.


In [4]:
model = keras.Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation= 'relu', input_shape=(224,224,3)))
model.add(Conv2D (64, (3,3), activation= 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout (0.1))

model.add(Conv2D(64, (3,3), activation= 'relu'))
model.add(MaxPooling2D (pool_size=(2,2)))
model.add(Dropout (0.1))

model.add(Conv2D (128, (3,3), activation= 'relu'))
model.add(MaxPooling2D (pool_size=(2,2)))
model.add(Dropout (0.1))

model.add(Flatten())

model.add(Dense (64, activation='relu'))
model.add(Dropout (0.5))
model.add(Dense (2, activation='softmax'))

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-5), metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 110, 110, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                    

In [6]:
train_datagen = image.ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = image.ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = image.ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_dir = '../train_test_val_split_images/train/'
test_dir = '../train_test_val_split_images/test/'
val_dir = '../train_test_val_split_images/validation/'

image_size = (224, 224)
batch_size = 32

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 64 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [8]:
print("Train classes: ", train_generator.class_indices)
print("Test classes: ", test_generator.class_indices)
print("Validation classes: ", val_generator.class_indices)

Train classes:  {'Meditation': 0, 'Thinking': 1}
Test classes:  {'Meditation': 0, 'Thinking': 1}
Validation classes:  {'Meditation': 0, 'Thinking': 1}


In [9]:
running_model = model.fit(
    train_generator,
    steps_per_epoch = 8,
    epochs = 10,
    validation_data = val_generator,
    validation_steps = 4
)

Epoch 1/10
8/8 [==============================] - 7s 592ms/step - loss: 0.6967 - accuracy: 0.4844 - val_loss: 0.6923 - val_accuracy: 0.5000
